In [2]:
import numpy as np
import pandas as pd
from pprint import pprint
import yfinance as yf
from scipy.optimize import minimize
import random

In [3]:
tickers=["AAPL","MSFT","AMZN","GOOGL"]
num_stocks=len(tickers)

cl_price=pd.DataFrame()

In [4]:
for ticker in tickers:
    cl_price[ticker]=yf.download(ticker,period="1y",interval="1d")["Adj Close"]

cl_price.to_csv("Closing_price.csv")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:
returns = cl_price.pct_change().dropna()
expected_returns = returns.mean()*252
covariance_matrix = returns.cov()*252
num_particles = 50
num_dimensions = num_stocks  # Each dimension corresponds to an asset weight
max_iterations = 100
inertia_weight = 0.5  # Weight for the particle's current velocity
cognitive_weight = 1.5  # Weight for the particle's best known position
social_weight = 1.5     # Weight for the swarm's best known position

def fitness_function(weights):
    portfolio_return = np.sum(expected_returns * weights)
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_risk
    return -sharpe_ratio  

particles = []
best_positions = []
best_fitness_values = []
for _ in range(num_particles):
    particle = np.random.uniform(0, 1, num_stocks)   # Initialize a random portfolio allocation
    particle /= np.sum(particle)  # Normalize to ensure weights sum to 1
    particles.append(particle)
    best_positions.append(particle) # Initialize the best-known position as the current position
    fitness_value = fitness_function(particle)   # Evaluate the fitness of the initial position
    best_fitness_values.append(fitness_value)

global_best_position = best_positions[np.argmin(best_fitness_values)]
global_best_fitness = min(best_fitness_values)
for iteration in range(max_iterations):
    for i in range(num_particles):
        # Update the particle's velocity
        inertia_term = inertia_weight * np.random.uniform(0, 1, num_dimensions) * (particles[i] - particles[i])
        cognitive_term = cognitive_weight * np.random.uniform(0, 1, num_dimensions) * (best_positions[i] - particles[i])
        social_term = social_weight * np.random.uniform(0, 1, num_dimensions) * (global_best_position - particles[i])
        velocity = inertia_term + cognitive_term + social_term
        # Update the particle's position
        particles[i] += velocity
        particles[i] = np.maximum(0, particles[i])  # Ensure non-negative weights
        particles[i] /= np.sum(particles[i])  # Normalize weights
        
        fitness_value = fitness_function(particles[i]) # Evaluate the fitness of the new position
        if fitness_value < best_fitness_values[i]:     # Update the particle's best-known position and fitness value
            best_positions[i] = particles[i]
            best_fitness_values[i] = fitness_value
        
        if fitness_value < global_best_fitness:        # Update the swarm's best-known position and fitness value
            global_best_position = particles[i]
            global_best_fitness = fitness_value
        

print("Optimal Asset Allocations using Particle Swarm:")
for i, symbol in enumerate(tickers):
    print(f"{symbol}: {global_best_position[i]:.4f}")

best_portfolio_return = np.sum(expected_returns * global_best_position)
best_portfolio_risk = np.sqrt(np.dot(global_best_position.T, np.dot(covariance_matrix, global_best_position)))
sharpe_ratio= best_portfolio_return/best_portfolio_risk

print(f"Expected Portfolio Return: {best_portfolio_return:.4f}")
print(f"Portfolio Risk (Standard Deviation): {best_portfolio_risk:.4f}")
print(f"Sharpe Ratio : {sharpe_ratio:.4f}")

Optimal Asset Allocations using Particle Swarm:
AAPL: 0.4639
MSFT: 0.2471
AMZN: 0.2518
GOOGL: 0.0372
Expected Portfolio Return: 0.4968
Portfolio Risk (Standard Deviation): 0.2073
Sharpe Ratio : 2.3963
